Calibration ignores hcal component and tries to linear fit, so one calibration constant is gained - a linear coefficient for the full ecal assuming a factor of 2 between thick and thin layers

Backscatter removed, no removed tails

In [1]:
using LCIO
using Plots
using Glob
using LsqFit
using StatsBase
using Distributions
gr();

In [2]:
# read the files for calibration
fileList = readdir(glob"reco_5000a*0phi*", "singleParticles")
phi=0.0
# build the list of input energies
energyString = r"_(\d+)GeV"
# get (match) the energy string from the filename, convert (parse) from string to Int16, build a dictionary from ints to filenames
energyMap = Dict((parse(Int16, match(energyString, fn)[1]), fn) for fn in fileList)
for (energy, filename) in energyMap
    println(energy, " GeV:\t", filename)
end

20 GeV:	singleParticles/reco_500a_20GeV_0phi.slcio
100 GeV:	singleParticles/reco_500a_100GeV_0phi.slcio
10 GeV:	singleParticles/reco_500a_10GeV_0phi.slcio
50 GeV:	singleParticles/reco_500a_50GeV_0phi.slcio
2 GeV:	singleParticles/reco_500a_2GeV_0phi.slcio
5 GeV:	singleParticles/reco_500a_5GeV_0phi.slcio
1 GeV:	singleParticles/reco_500a_1GeV_0phi.slcio


In [3]:
function getHitsFromFile(filename)
    eCalEnergies = Float64[]
    eCalLengths = Int16[]
    eCalBackEns = Float64[]
    LCIO.open(filename) do reader
        for (idx, event) in enumerate(reader)
            if idx > 10000
                break
            end
            eCalEnergy = 0.0
            eCalLength = 0
            eCalBackEn = 0.0
            # sum up the uncalibrated ECalHits
            # this needs to be sorted by layer, so we need a decoder
            EcalBarrelHits = getCollection(event, "ECalBarrelHits")
            decode = CellIDDecoder(EcalBarrelHits)
            for hit in EcalBarrelHits
                ##eliminate backscatter - if angle > 0.2 rad from center at "phi"
                angle=(acos((cos(phi*pi/180.)*getPosition(hit)[1]+sin(phi*pi/180.)*getPosition(hit)[2])/sqrt(getPosition(hit)[1]*getPosition(hit)[1]+getPosition(hit)[2]*getPosition(hit)[2]+getPosition(hit)[3]*getPosition(hit)[3])))
                if getPosition(hit)[1]<0
                    angle=-angle
                end
                if phi*pi/180+0.2>angle>phi*pi/180-0.2
                    # calibrate the hits in the later layers with a higher number, because they are behind thicker tungsten slabs
                    factor = decode(hit)["layer"] < 21 ? 1 : 2
                    if decode(hit)["layer"]>0
                        eCalEnergy += factor*getEnergy(hit)
                        eCalLength += 1
                        if decode(hit)["layer"]>=29
                            eCalBackEn+=getEnergy(hit)
                        end
                    end
                end
            end
            # fixme: Simple outlier cut
            if eCalLength < 30#100
                continue
            end
            push!(eCalEnergies, eCalEnergy)
            push!(eCalLengths, eCalLength)
            push!(eCalBackEns, eCalBackEn)
        end
    end
    return eCalEnergies, eCalLengths, eCalBackEns
end;

In [4]:
eHits  = Dict{Int16, Vector{Float64}}()
eHitsBack  = Dict{Int16, Vector{Float64}}()
eCount = Dict{Int16, Vector{Int16}}()
for (energy, filename) in energyMap
#    if energy < 10 
#        continue
#    end
#    if energy != 1
#        continue
#    end
    println("Processing file for ", energy, " GeV")
    eCal, nEhits, eCalBack= getHitsFromFile(filename)
    eHits[energy] = eCal
    eHitsBack[energy]= eCalBack
    eCount[energy] = nEhits
end

Processing file for 20 GeV
Processing file for 100 GeV
Processing file for 10 GeV
Processing file for 50 GeV
Processing file for 2 GeV
Processing file for 5 GeV
Processing file for 1 GeV


In [5]:
# removes the 10% of the furthest outliers on either side
# no assumption about smoothness
function removeTails(distribution, cutOff=10)
    sort!(distribution)
    l = length(distribution)
    lcut = round(Int64, l * cutOff/100)
    hcut = round(Int64, l * (100-cutOff)/100)
    # start out with the whole distribution
    minDist = distribution[end] - distribution[1]
    low = 1
    high = l
    for idx = 1:lcut
        dist = distribution[hcut+idx] - distribution[idx]
        if dist < minDist
            minDist = dist
            low = idx+1
            high = hcut+idx
        end
    end
    return low, high
end;

In [6]:
histogram([eHits[energy] for energy in keys(eHits)], fillalpha=0.5, linewidth=0, label=map(string, keys(eHits)))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 100 
 
 
 
 
 10 
 
 
 
 
 50 
 
 
 
 
 2 
 
 
 
 
 5 
 
 
 
 
 20

In [7]:
histogram([eHitsBack[energy] for energy in keys(eHits)], fillalpha=0.5, linewidth=0, label=map(string, keys(eHits)),xlabel="GeV")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.000 
 
 
 0.025 
 
 
 0.050 
 
 
 0 
 
 
 100 
 
 
 200 
 
 
 300 
 
 
 400 
 
 
 GeV 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 100 
 
 
 
 
 10 
 
 
 
 
 50 
 
 
 
 
 2 
 
 
 
 
 5 
 
 
 
 
 20

In [8]:
# this function attempts a global fit and minimizes the offset b of the quadratic form y=axx+mx+b
# parameters are ecal energies (×2 for the hits in the outer layers), hcal energies, particle energy
function lineFitter(ecal, truValues)
    function model(x, p)
        energies = Dict{Int16, Float64}()
        sigma = 0.0
        for e in truValues
            calibrated = p[1] .* x[e] #+ p[2] .* x[e] .* x[e]
            calibrated1=p[1].*x[e]
            # cut the tails, fit a Normal distribution to the result
  #          low, high = removeTails(calibrated)
  #          n = Distributions.fit(Normal, calibrated[low:high])
            n=Distributions.fit(Normal,calibrated)
            energies[e] = n.μ
            sigma=n.σ
        end
        # we are optimizing for the ratio of reconstructed energies to true values
#        println("sigma = ",sigma)#prints largest sigma value
        return [energies[e]/e for e in truValues]
    end
    fit = curve_fit(model, ecal, 1.0, [50.0])
    errors=estimate_errors(fit,0.95)
#    println(fit.param,errors)
    return fit.param,errors
end
ECal, ECalErr = lineFitter(eHits, keys(eHits))
println("ECal calibration constant: ", ECal, " +/- ", ECalErr)

ECal calibration constant: [58.7852] +/- [0.309724]


In [9]:
histogram([ECal .* eHits[energy]  for energy in keys(eHits)], fillalpha=0.5, linewidth=0, label=map(string, keys(eHits)))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 0 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1 
 
 
 
 
 100 
 
 
 
 
 10 
 
 
 
 
 50 
 
 
 
 
 2 
 
 
 
 
 5 
 
 
 
 
 20

In [10]:
histogram([ECal .* eHits[100]], fillalpha=0.5, linewidth=0)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 95 
 
 
 100 
 
 
 105 
 
 
 0 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1

In [11]:
y = Vector{Float64}()
yerr = Vector{Float64}()
x = Vector{Float64}()
xLin=[1.,2.,5.,10.,20.,50.,100.]
for e in keys(eHits)
    d = ECal .* eHits[e] 
#    low, high = removeTails(d)
#    gauss = Distributions.fit(Normal, d[low:high])
    gauss=Distributions.fit(Normal,d)
    push!(y, gauss.μ)
    push!(yerr, gauss.σ)
    push!(x, 1.0*e)
end
println("100 GeV distribution: μ = ",y[2]," σ = ", yerr[2])
println("10 GeV distribution: μ = ",y[3]," σ = ", yerr[3]) #array order: 1, 100, 10, 50, 2, 5, 20
println("1 GeV distribution: μ = ",y[1]," σ = ", yerr[1])
line(x, p) = p[1] + p[2]*x
l = curve_fit(line, x, y, [0.0, 1.0])
plot(x, y, yerr=yerr, marker=stroke(2), line=false, label="data",xlabel="Initial Energy [GeV]",ylabel="Calibrated Energy [GeV]")
leg = @sprintf("y=%.2f + %.2f*x", l.param[1], l.param[2])
plot!(x, l.param[1]+l.param[2]*x, label=leg)
plot!(xLin,xLin,label="Diagonal")

100 GeV distribution: μ = 99.40928348051183 σ = 2.1862417462109853
10 GeV distribution: μ = 9.962500627766687 σ = 0.7121751490312089
1 GeV distribution: μ = 1.014265890830537 σ = 0.17586900302676076


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 Initial Energy [GeV] 
 
 
 Calibrated Energy [GeV] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 data 
 
 
 
 y=0.04 + 0.99*x 
 
 
 
 Diagonal

In [12]:
yyy = Vector{Float64}()
yyyerr = Vector{Float64}()
for e in keys(eHits)
    d = (ECal .* eHits[e])./ (1.0.*e)
#    low, high = removeTails(d)
#    gauss = Distributions.fit(Normal, d[low:high])
    gauss=Distributions.fit(Normal,d)
    push!(yyy, gauss.μ)
    push!(yyyerr, gauss.σ/sqrt(500))
end

plot(x, yyy, yerr=yyyerr, marker=stroke(2), line=false, leg=false,xlabel="Initial Energy [GeV]",ylabel="Calibrated Energy / Initial Energy")
xaxis!(:log10,(0.9,110.0))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 0 
 
 
 10 
 
 
 1 
 
 
 10 
 
 
 2 
 
 
 1.00 
 
 
 1.01 
 
 
 1.02 
 
 
 Initial Energy [GeV] 
 
 
 Calibrated Energy / Initial Energy

In [13]:
yy = Vector{Float64}()
yyerr = Vector{Float64}()
for e in keys(eHits)
    d = eHits[e] 
#    low, high = removeTails(d)
#    gauss = Distributions.fit(Normal, d[low:high])
    gauss=Distributions.fit(Normal,d)
    push!(yy, gauss.μ)
    push!(yyerr, gauss.σ)
end

ll = curve_fit(line, x, yy, [0.0, 1.0])
plot(xLin,xLin,label="Diagonal")
plot!(x, yy, yerr=yyerr, marker=stroke(1), line=false, label="data-uncalibrated",xlabel="Initial Energy [GeV]",ylabel="Raw (uncalibrated) Energy [GeV]")
legg = @sprintf("y=%.2f + %.2f*x", ll.param[1], ll.param[2])
plot!(x, ll.param[1]+ll.param[2]*x,label=legg)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 Initial Energy [GeV] 
 
 
 Raw (uncalibrated) Energy [GeV] 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Diagonal 
 
 
 
 
 
 
 
 data-uncalibrated 
 
 
 
 y=0.00 + 0.02*x

In [14]:
zzz = Vector{Float64}()
zzzerr = Vector{Float64}()
for e in keys(eHits)
    d = 100.0.*(1.0-((ECal .* eHits[e])./ (1.0.*e)))
#    low, high = removeTails(d)
#    gauss = Distributions.fit(Normal, d[low:high])
    gauss=Distributions.fit(Normal,d)
    push!(zzz, gauss.μ)
    push!(zzzerr, gauss.σ./sqrt(2000.))
end

plot(x, zzz, yerr=zzzerr, marker=stroke(2), line=false, leg=false,xlabel="Initial Energy [GeV]",ylabel="1 - (Calibrated Energy / Initial Energy) [%]")


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 -1 
 
 
 0 
 
 
 Initial Energy [GeV] 
 
 
 1 - (Calibrated Energy / Initial Energy) [%]

In [15]:
zzzz = Vector{Float64}()
zzzzerr = Vector{Float64}()
for e in keys(eHits)
    d = ((1.0.*e)-(ECal .* eHits[e]))
#    low, high = removeTails(d)
#    gauss = Distributions.fit(Normal, d[low:high])
    gauss=Distributions.fit(Normal,d)
    push!(zzzz, gauss.μ)
    push!(zzzzerr, gauss.σ./sqrt(2000.))
end

plot(x, zzzz, yerr=zzzzerr, marker=stroke(2), line=false, leg=false,xlabel="Initial Energy [GeV]",ylabel="Initial Energy - Calibrated Energy [Gev]")
yaxis!((-0.22, 0.65))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 50 
 
 
 100 
 
 
 0.00 
 
 
 0.25 
 
 
 0.50 
 
 
 Initial Energy [GeV] 
 
 
 Initial Energy - Calibrated Energy [Gev]